# Part 0 - The Kaplan-Meyer Estimator

Lets first familiarize ourselves with the Kaplan–Meier estimator. You might remember that its formula reads


\begin{equation*}
\widehat S(t) = \prod_{i:\ t_i\le t} \left( 1 - \frac{d_i}{n_i} \right)
\end{equation*}


Where $\widehat S(t)$ is the estimator for time $t$, $d_i$ the number of deaths at time $t_i$ and $n_i$ the number of individuals that are **known to have survived** until time $t_i$.

With this in mind, use the following data to calculate $\widehat S(t)$ and sketch a plot for the estimator.

|Subject|Survival Time|Status|
|---|---|---|
|Fly 1|1 day|Dead|
|Fly 2|2 days|Alive|
|Fly 3|3 days|Alive|
|Fly 4|4 days|Dead|
|Fly 5|5 days|Dead|
|Fly 6|6 days|Alive|
|Fly 7|7 days|Dead|
|Fly 8|8 days|Dead|
|Fly 9|9 days|Alive|
|Fly 10|10 days|Alive|



---

# Part 1 - Importing necessary packages

This will import the packages we will use during the workshop. If the installation was done correctly, there should be no errors here.

In [ ]:
%matplotlib inline

# Numerical library
import numpy as np

# Data manipulation
import pandas as pd
from patsy import dmatrix

# Ploting
import matplotlib
import matplotlib.pyplot as plt

# Survival analysis
import lifelines

from IPython.display import *

# Part 2 - A look at the clinical data

The data is originally available at http://www.cbioportal.org/study?id=brca_tcga_pub, but it can also be found together with this notebook at https://github.com/gjeuken/survival-workshop.

We will use here clinical data on the patients as well as gene expression data. This is data from real experiments (!) so you might even discover something that went unnoticed until now.

Firstly, let us load the clinical data and see what it looks like. Make a note of all the variables (columns) available in the data.

In [ ]:
clinical = pd.read_csv('data/brca_tcga_pub_clinical_data.tsv', sep='\t')

with pd.option_context('display.max_columns', None):
    display(clinical)

To make the later survival analysis easier, we create two new columns, one for the survival in days, and one for the event (death) being present.

In [ ]:
clinical['Time'] = clinical['Overall Survival (Months)']*30
clinical['Event'] = (clinical['Overall Survival Status'] == 'DECEASED')*1

Now, let us plot how the survival data looks like

In [ ]:
matplotlib.rcParams['figure.figsize'] = [15, 50]

data_sorted = clinical[['Time','Event']].sort_values(by = 'Time').dropna().reset_index(drop=True)
print(data_sorted.shape)
status_slice = data_sorted['Event'] == 1

fig, ax = plt.subplots()

ax.barh(data_sorted.loc[~status_slice].index, data_sorted.loc[~status_slice,'Time'], height = 1, color = 'b')
ax.barh(data_sorted.loc[status_slice].index, data_sorted.loc[status_slice,'Time'], height = 1, color = 'r')

ax.set_xlabel('Days alive')
ax.set_ylabel('Patients')
ax.legend(['Alive', 'Dead'])
ax.xaxis.tick_top()
ax.grid()

---
### Exercises
2.1 Describe the plot and what inferences you are able to make from it

---
# Part 3 - Survival analysis using the clinical data



Ok, now that we've seen the data, let's play around with it.

How does the survival curve look like in general? We can use the survival package __lifelines__ to figure this out, and generate a *Kaplan Meier plot*. 

We will also compare the Kaplan Meier estimate for $S(t)$ with a naive estimate.

In [ ]:
kmf = lifelines.KaplanMeierFitter()
kmf_naive = lifelines.KaplanMeierFitter()

clinical = clinical.dropna()

kmf.fit(clinical['Time'], clinical['Event'])
kmf_naive.fit(clinical['Time'], np.repeat(1, len(clinical['Time'])))

matplotlib.rcParams['figure.figsize'] = [15, 10]
ax = kmf.plot(ci_show=False)
kmf_naive.plot(ax=ax, ci_show=False)
plt.legend(['Kaplan-Meier estimate','Naive estimate'])
plt.xlabel('Days')
plt.ylabel('$S(t)$')
plt.ylim([0,1])


---
### Exercises

3.1 Why are both estimates so different? Can you relate your answer to the first plot and the data?

3.2 Compare the Kaplan Meier plot with the first one, what additional insights are available on this latter plot?

3.3 What are the common features of the two plots?

---

Now we can start to play around with clinical variables that might influence in the survival curve.

You should play around with the groupings and see if you can find some useful insight

In [ ]:
### Define groups here
group_1 = clinical.loc[clinical['Metastasis-Coded'] == 'Positive']
group_2 = clinical.loc[clinical['Metastasis-Coded'] == 'Negative']
###

kmf.fit(group_1['Time'], group_1['Event'], label='Positive')
ax = kmf.plot(ci_show=False)

kmf.fit(group_2['Time'], group_2['Event'], label='Negative')
kmf.plot(ax=ax, ci_show=False)

plt.title('Metastasis')

---
### Exercises
3.4 Make at least 3 plots using relevant separation criteria, and label them accordingly. Why did you choose these criteria? Are the results as expected?

3.5 Make a plot that includes the survival curve for 3 different age groups. Do the results make sense?

3.6 (advanced) The [**logrank_test** function](http://lifelines.readthedocs.io/en/latest/Examples.html?compare-using-a-hypothesis-test#compare-using-a-hypothesis-test) of the **lifelines** package performs a statistical test on the two groups to see if their event (death) generation process is the same. Use this function to obtain a significance statistic for the separations above.

---

# Part 4 - Cox Proportional Hazards model

We will now introduce the Cox Proportional Hazards (CPH) model. To do so, lets first define the hazard function:

\begin{equation*}
\lambda(t) = \lim_{dt \rightarrow 0} \frac{\Pr(t \leq T < t+dt)}{dt\cdot S(t)} = - \frac{S'(t)}{S(t)}
\end{equation*}

To understand the model, we first need to understand its assumptions. The main assumption here (proportional hazards condition) states that covariates are multiplicatively related to the hazard, or in other words, a change in the covariate will have an effect on the hazard rate that is proportional to the magnitude of that change.

Now we can in introduce the model. The formula for CPH reads:

\begin{equation*}
\lambda(t|X_i) = \lambda_0(t)\exp(\beta_1X_{i1} + \cdots + \beta_pX_{ip}) = \lambda_0(t)\exp(X_i \cdot \beta)
\end{equation*}

Where $X_i = \{X_{i,1}, X_{i,2}, \dots, X_{i,p}\}$ are the realized values of the covariates for subject $i$, $\lambda_0(t)$ is the *baseline* hazard rate, and $\beta_i$ are the coefficients of proportionality.
With a careful read, you can see how this equation relates to the assumption discussed. If you want to read more about the model, and especially how the coefficients are calculated, a great place to start is the [Wikipedia page](https://en.wikipedia.org/wiki/Proportional_hazards_model).

The next step is to use the Cox Proportional Hazards regression in the data. 
First, let's check the main assumption of the model, the proportionality of hazards, by plotting the hazard functions on a log-log scale and seeing whether they "look" parallel or not. We will do this for a few covariates.

In [ ]:
### Define groups here
group_1 = clinical.loc[clinical['Metastasis-Coded'] == 'Positive']
group_2 = clinical.loc[clinical['Metastasis-Coded'] == 'Negative']
###

kmf.fit(group_1['Time'], group_1['Event'], label='Positive')
ax = kmf.plot_loglogs()

kmf.fit(group_2['Time'], group_2['Event'], label='Negative')
kmf.plot_loglogs(ax=ax)

---
### Exercises

4.1 Can you explain what the baseline hazard means?

4.2 Why can we test our hypothesis by looking at the plot above?

4.3 In your opinion, does the proportional hazards assumption hold for the Metastasis case? What about other variables you used in the previous exercise? Use the plots to support your claims.

---

Now that we checked the proportional hazards assumption, we can perform the CPH regression on the categorical data where this assumption holds.

Note that here we will use the [patsy package](https://patsy.readthedocs.io/en/latest/) to format our categorical data in a way that is suited for the regression. If you are not familiar with this type of representation, please take a moment to familiarize yourself in the [relevant help page](https://patsy.readthedocs.io/en/latest/categorical-coding.html).




In [ ]:
cph = lifelines.CoxPHFitter()

formula = 'Time + Event + C(Q("Metastasis"), levels = l) + Q("Diagnosis Age")'

l = ['M0', 'M1']


cox_input = dmatrix(formula, data = clinical, return_type = 'dataframe')
cox_input = cox_input.drop(['Intercept'], axis = 1)

cph.fit(cox_input, duration_col='Time', event_col='Event')

cph.plot()
cph.summary

---
### Exercises
4.4 For the regression performed above (Metastasis + Age), compare both coefficients. What can you learn from them?

4.5 Perform the CPH regression using a few variables where the proportional hazard assumption hold.

4.6 Try adding more than one variable in the same regression. What happens then? Explain the results.

4.7 (advanced) If there are variables where the proportional hazard assumption does not hold, but hold predictive power, you might still use it for <a href="https://en.wikipedia.org/wiki/Blocking_(statistics)">blocking</a>. How do the results change then? [Help](http://lifelines.readthedocs.io/en/latest/Survival%20Regression.html#stratification)

4.8 Can you find a way to plot the survival curves predicted by the regression for each level of a categorical variable? (Remember the CPH assumptions)

4.9 What are the difference in the results compared to the Kaplan-Mayer statistics when using the same covariates?

---

# Part 5 - Incorporating gene expression data

It's time to look at the gene expression data! This data is generated using RNASeq on tumor tissue. 

First, we load the expression data for the same samples. This may take some time, be patient.

We then take a look at how the data looks like. Note that the gene expression has been median normalized, so we get negative gene expression here.

In [ ]:
expression_raw  = pd.read_csv('data/data_expression_median.txt', sep='\t')
expression = expression_raw.set_index('Hugo_Symbol').iloc[:,1:].T
expression

Now we merge the clinical and expression data in one big data structure, and display the result.

In [ ]:
data = clinical.merge(expression, how='inner', left_on='Sample ID', right_index=True)
data

---
### Exercises

5.1 How many genes are there in the data? Is there any gene you think might be relevant for the survival time in breast cancer?

5.2 Can you think of natural ways to incorporate this data on both the Kaplan–Meier estimator and the Cox Proportional Hazards models?

----

Let's incorporate the data in our study. We will start with the Kaplan–Meier estimator, and to do so, we will have to separate our subjects in groups according to gene expression levels.
We will next plot the Kaplan–Meier estimates, together with the results of the statistical test.


In [ ]:
### Define groups here
treshold = 0

group_1 = data.loc[data['BRCA1'] > treshold]
group_2 = data.loc[data['BRCA1'] <= treshold]
###

kmf.fit(group_1['Time'], group_1['Event'], label='Positive')
ax = kmf.plot(ci_show=False)

kmf.fit(group_2['Time'], group_2['Event'], label='Negative')
kmf.plot(ax=ax, ci_show=False)

plt.title('BRCA1')

from lifelines.statistics import logrank_test

test = logrank_test(group_1['Time'], group_2['Time'], event_observed_A=group_1['Event'], event_observed_B=group_2['Event'])
test.summary

---
### Exercises

5.3 What criteria would you use to separate the groups using the gene expression information? Can you come up with a few?

5.4 Perform the calculation with some genes of your choice, and using a few separation criteria. Do you get interesting results? Are they statistically significant?

---

We can also do the same for the CPH model. Let's incorporate the gene expression data into our Cox regression.

Notice that here there is no need to separate the subjects into groups.

In [ ]:
formula = 'Time + Event + BRCA1'

cox_input = dmatrix(formula, data = data, return_type = 'dataframe')
cox_input = cox_input.drop(['Intercept'], axis = 1)

cph.fit(cox_input, duration_col='Time', event_col='Event')

cph.plot()
cph.summary

---
### Exercises
5.5 Perform the CPH regression with some genes of your choice. Did you find any gene that has a significant influence in the survival rate for breast cancer?

5.6 How do the results differ from the test performed previously?

5.7 (advanced) You may want to write a code that automatically tests all the genes and outputs all the relevant genes and their regression coefficients in the end.

5.8 Now try multiple combinations of the genes in the same regression, start with the genes found in the previous exercise. Again, do the results change? Why?



# The final exercise

What is your opinion of survival analysis in general? How do you compare both models studied here? What are their merits and disadvantages? Where would you use one instead of the other?






----
**Thank you for participating!** Any feedback on the content and delivery of the workshop is welcomed at gustavo.jeuken@scilifelab.se.
